대학, 학과, 교번, 성명, 풀네임, 약어, Scopus ID, Scopus ID 갯수, PNU ID, 직급, 이메일, 임용일자, 퇴직일자

In [6]:
import pandas as pd
import numpy as np
from pandas import DataFrame as df
from openpyxl import load_workbook

pd.options.display.float_format = '{:.0f}'.format

jeunyim_list = pd.read_excel("연구자명단/교무과제공_전임교원 명단_임용일자추가(2020.4.27.기준).xlsx")
giegum_list = pd.read_excel("연구자명단/교무과제공_기금교수 명단_임용일자추가(2020.4.20.현재기준).xlsx")
profess_list = pd.read_excel("연구자명단/도서관제공_교수급 연구자명단(원본).xlsx")
pip_all = pd.read_excel("PIP_ALL.xlsx")

jeunyim_cnt = len(jeunyim_list)
giegum_cnt = len(giegum_list)

Researcher_temp_list = df(columns=["대학","학과","성명","영문풀네임","영문약어","직급","교번","PNU_ID","SCOPUS_ID"])
Researcher_final_list = df(columns=["대학","학과","성명","영문풀네임","영문약어","직급","교번","PNU_ID","SCOPUS_ID"])

# 전임교원 명단 추가
for pro in range(0, jeunyim_cnt):
    res_number = jeunyim_list.loc[pro][5] #교번은 유일
    if res_number in profess_list['교번'].tolist():
        index_num = profess_list['교번'].tolist().index(res_number)
    else:
        index_num=40
    
    Researcher_temp_list.loc[pro] = [
        jeunyim_list.loc[pro][1], #대학
        jeunyim_list.loc[pro][2], #학과
        jeunyim_list.loc[pro][4], #성명
        profess_list.loc[index_num][4], #영문풀네임 from 도서관
        profess_list.loc[index_num][5], #영문약어 from 도서관
        jeunyim_list.loc[pro][3], #직급
        jeunyim_list.loc[pro][5], #교번
        profess_list.loc[index_num][0], #PNU_ID from 도서관
        "NONE"#jeunyim_list.loc[pro][0], #SCOPUS_ID
    ]
    
# 기금교원 명단 추가
for pro in range(0,giegum_cnt):
    res_number = giegum_list.loc[pro][4] #교번은 유일
    if res_number in profess_list['교번'].tolist():
        index_num = profess_list['교번'].tolist().index(res_number)
    else:
        index_num=40
    
    Researcher_temp_list.loc[jeunyim_cnt+pro] = [
        giegum_list.loc[pro][1], #대학
        giegum_list.loc[pro][2], #학과
        giegum_list.loc[pro][5], #성명
        profess_list.loc[index_num][4], #영문풀네임 from 도서관
        profess_list.loc[index_num][5], #영문약어 from 도서관
        giegum_list.loc[pro][3], #직급
        giegum_list.loc[pro][4], #교번
        profess_list.loc[index_num][0], #PNU_ID from 도서관
        "NONE"#jeunyim_list.loc[pro][0], #SCOPUS_ID
    ]
 
#print(Researcher_temp_list.shape)
#Researcher_temp_list
#Researcher_total_list.to_excel("researcher_table.xlsx")
# 학과명별로 재분류
pnu_department_list = Researcher_temp_list["학과"].unique()
tmp=[]
for department in pnu_department_list:
    tmp.append(Researcher_temp_list.groupby('학과').get_group(department).sort_values('성명', ascending=True))

for dep in range(0,len(tmp)):
    Researcher_final_list = pd.concat([Researcher_final_list,tmp[dep]])
    
#Researcher_final_list.to_excel("저자종합명단.xlsx")
Researcher_list_v1 = Researcher_final_list.reset_index(drop=True)
total_prf_cnt = len(Researcher_list_v1)

#Researcher_list_v1


In [11]:
Researcher_list_v3 = df(columns=["대학","학과","성명","교번","풀네임","약어",
                                 "Scopus_ID_1",
                                 "Scopus_ID_2",
                                 "Scopus_ID개수",
                                 "PIP_documents",
                                 "Scopus_documents",
                                 "Scopus_documents_5",
                                 "PNU_ID","직급","이메일","임용일","퇴직일"])


In [13]:
for pro in range(0,total_prf_cnt):
    
    kyo_num = Researcher_list_v1.loc[pro][6]
    email=None
    app_date=None
    ret_date=None
    
    #print(kyo_num)
    if kyo_num in profess_list['교번'].tolist():
        email=profess_list['email'].loc[profess_list['교번'].tolist().index(kyo_num)]
    if kyo_num in pip_all['교직원번호'].tolist():
        app_date=pip_all['임용일자'].loc[pip_all['교직원번호'].tolist().index(kyo_num)]
        ret_date=pip_all['퇴직일자'].loc[pip_all['교직원번호'].tolist().index(kyo_num)]
    
    Researcher_list_v3.loc[pro] = [
        Researcher_list_v1.loc[pro][0], # 대학
        Researcher_list_v1.loc[pro][1], # 학과
        Researcher_list_v1.loc[pro][2], # 성명
        kyo_num, # 교번
        Researcher_list_v1.loc[pro][3], # 풀네임
        Researcher_list_v1.loc[pro][4], # 약어
        
        None,#Researcher_list_v1.loc[pro][], # Scopus id-1
        None,#Researcher_list_v1.loc[pro][], # Scopus id-2
        None,#Researcher_list_v1.loc[pro][], # Scopus id number
        None,#Researcher_list_v1.loc[pro][], # Scopus pip_documents
        None,#Researcher_list_v1.loc[pro][], # Scopus scopus_documents
        None,#Researcher_list_v1.loc[pro][], # Scopus scopus_documents5
        
        Researcher_list_v1.loc[pro][7], # pnu id
        Researcher_list_v1.loc[pro][5], # 직급
        email, # 이메일
        app_date, # 임용일
        ret_date, # 퇴직일
    ]
                        
Researcher_list_v3.to_excel("저자종합명단.xlsx")

In [15]:
Researcher_list_v3.loc[0]

대학                                   공과대학
학과                                  전기공학과
성명                     KANDASAMY PRABAKAR
교번                                 111424
풀네임                    KANDASAMY PRABAKAR
약어                            Prabakar, K
Scopus_ID_1                          None
Scopus_ID_2                          None
Scopus_ID개수                          None
PIP_documents                        None
Scopus_documents                     None
Scopus_documents_5                   None
PNU_ID                               1100
직급                                     교수
이메일                   kprabakar@gmail.com
임용일                              20080307
퇴직일                                   NaN
Name: 0, dtype: object